In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

In [2]:
%load_ext nb_mypy

Version 1.0.5


In [3]:
from typing import TypeVar

# Alpha-Beta Pruning

This notebook implements [alpha-beta pruning](https://en.wikipedia.org/wiki/Alpha-beta_pruning). *Memoization techniques* are only added in a naive way since adding these techniques in a sophisticated way results in an algorithm that is quite complicated.  Therefore, this topic is
discussed in a separate notebook.

Effectively, this notebook is a *game solver* because it can be used to play various deterministic, zero-sum, two-person games with perfect information.  To this end, the implementation assumes that an external notebook defines a game and provides the following variables and functions:
* `gPlayers` is a list of length two.  The elements of this list are the 
  players.  It is assumed that the first element in this list represents 
  the computer, while the second element is the human player.  The computer
  always starts the game.
* `gStart` is the start state of the game.
* `next_states(state, player)` is a function that takes two arguments:
  - `state` is a state of the game.
  - `player` is the player whose turn it is to make a move.
  The function call `next_states(state, player)` returns the list
  of all states that can be reached by any move of `player`.
* `utility(state)` takes a `state` and a `player` as its arguments.
  If `state` is a *terminal state*, then the function returns the value
  that this `state` has for the player who starts the game.  Otherwise, the function returns `None`.
* `finished(state)` returns `True` if and only if `state` is a terminal state.
* `get_move(state)` displays the given state and asks the human player for
  her move.
* `final_msg(state)` informs the human player about the result of the game.
* `draw(state, canvas, value)` draws the given state on the canvas and informs
  the user about the `value` of this state. 
   
---

In [4]:
State = TypeVar('State')

In [5]:
import ipycanvas as cnv

In [6]:
gPlayers: list[str]
    
def next_states(S: State, player: str) -> list[State]: 
    return None # type: ignore

def utility(S: State) -> int | None:
    return None

def finished(S: State) -> bool: 
    return None # type: ignore

def get_move(S: State) -> State:
    return None # type: ignore

def final_msg(S: State) -> bool:
    return None # type: ignore

def toString(S: State) -> str:
    return None # type: ignore

def draw(S: State, canvas: cnv.canvas, value: str) -> None:
    return None

def alphaBetaMax(S: State, alpha: int, beta: int) -> int:
    return None # type: ignore

def alphaBetaMin(S: State, alpha: int, beta: int) -> int:
    return None # type: ignore

In order to have some variation in our game, we use random numbers to choose between optimal moves.

In [7]:
import random
random.seed(0)

The function `alphaBetaMax` takes three arguments:
- `State` is the current state of the game,
- `alpha` is a lower bound,
- `beta` is an upper bound.

The function `max_Value` returns the *value* that the given `State` has for the computer 
if both players play their optimal game.  This value is an element from the set $\{-1, 0, 1\}$.  
- If the first player can force a win, the return value is `1`.
- If the first player can at best force a draw, the return value is `0`.
- If the second player can force a win, the return value is `-1`.

Given that $\texttt{value}(s)$ is the value of the state $s$, the function `alphaBetaMax` satisfies the following *specification*:
- $\alpha \leq \texttt{maxValue}(s) \leq \beta \;\rightarrow\;\texttt{alphaBetaMax}(s, \alpha, \beta) = \texttt{maxValue}(s)$
- $\texttt{maxValue}(s) < \alpha \;\rightarrow\; \texttt{alphaBetaMax}(s, \alpha, \beta) \leq \alpha$
- $\beta < \texttt{maxValue}(s) \;\rightarrow\; \beta \leq \texttt{alphaBetaMax}(s, \alpha, \beta)$

Note that this *specification* is not a *definition* of the function
`alphaBetaMax` as there are many functions that satisfy this specification.  

In [8]:
def alphaBetaMax(S: State, alpha: int, beta: int) -> int:
    if finished(S):
        return utility(S) # type: ignore
    for ns in next_states(S, gPlayers[0]):
        value = alphaBetaMin(ns, alpha, beta)
        if value >= beta:
            return value
        alpha = max(alpha, value)
    return alpha

Given that $\texttt{minValue}(s)$ is the value of the state $s$, the function `alphaBetaMin` satisfies the following specification:
- $\alpha \leq \texttt{minValue}(s) \leq \beta \;\rightarrow\;\texttt{alphaBetaMin}(s, \alpha, \beta) = \texttt{minValue}(s)$
- $\texttt{minValue}(s) < \alpha \;\rightarrow\; \texttt{alphaBetaMin}(s, \alpha, \beta) \leq \alpha$
- $\beta < \texttt{minValue}(s) \;\rightarrow\; \beta \leq \texttt{alphaBetaMin}(s, \alpha, \beta)$

In [9]:
def alphaBetaMin(S: State, alpha: int, beta: int) -> int:
    if finished(S):
        return utility(S) # type: ignore
    for ns in next_states(S, gPlayers[1]):
        value = alphaBetaMax(ns, alpha, beta)
        if value <= alpha:
            return value
        beta = min(beta, value)
    return beta

The function `best_move` takes one argument:
- `State` is the current state of the game.

The function `best_move` returns a pair of the form $(v, s)$ where $s$ is a state and $v$ is the value of this state.  The state $s$ is a state that is reached from `State` if `player` makes one of her optimal moves.  In order to have some variation in the game, the function randomly chooses any of the optimal moves.

In [10]:
def best_move(S: State) -> tuple[int, State]:
    NS        = next_states(S, gPlayers[0])
    bestVal   = alphaBetaMax(S, -1, 1)
    bestMoves = [s for s in NS if alphaBetaMin(s, -1, 1) == bestVal]
    bestState = random.choice(bestMoves)
    return bestVal, bestState

The next line is needed because we need the function `IPython.display.clear_output` to clear the output in a cell.

In [11]:
import IPython.display 

The function `play_game` plays the game that is specified externally via the functions `next_states` and `utility`.  The states of the game are drawn on the given `canvas`. 

In [12]:
def play_game(canvas: cnv.canvas) -> None:
    State = gStart # type: ignore
    while (True):
        val, State = best_move(State);
        draw(State, canvas, f'For me, the game has the value {val}.')
        if finished(State):
            final_msg(State)
            return
        IPython.display.clear_output(wait=True)
        State = get_move(State)
        draw(State, canvas, '?')
        if finished(State):
            IPython.display.clear_output(wait=True)
            final_msg(State)
            break

In [13]:
%%capture
%run Nim-Frame.ipynb

In [14]:
%%capture
%run 2-Tic-Tac-Toe-Bitboard.ipynb

In [15]:
%unload_ext nb_mypy

If we use *$\alpha$-$\beta$ pruning*, computing the value of the `Start` state of *tic-tac-toe* takes about 52 ms.  This should be compared to the 0.6 seconds that the *minimax algorithm* needs for the bit-board implementation of *tic-tac-toe*.

In [16]:
%%time
val = alphaBetaMax(gStart, -1, 1)
val

CPU times: total: 31.2 ms
Wall time: 66.5 ms


0

## Naive Memoization

The implementation of `memoize` that is given below uses the fact that the functions `alphaBetaMax` and `alphaBetaMin` are never called with the same arguments.  Hence, we do not need to store the function `f`.

In [17]:
from typing import Callable, Any 

We define a generic type variable for the return type of an arbitrary function `f`.

In [18]:
R = TypeVar('R')

We define a `Args` as the generic type variable for the arguments of an arbitrary function `f`.

In [19]:
Args = tuple[Any, ...]

The keys of the dictionary `gCache`are the arguments,
and the values are the results of those function calls.

In [20]:
gCache: dict[tuple[Callable[..., R], Args], R] = {}

In [21]:
def memoize(f: Callable[..., R]) -> Callable[..., R]:
    global gCache
    
    def f_memoized(*args: Any) -> R:
        if args in gCache:
            return gCache[args]
        result = f(*args)
        gCache[args] = result
        return result
    
    return f_memoized

In [22]:
alphaBetaMax = memoize(alphaBetaMax)
alphaBetaMin = memoize(alphaBetaMin)

In [23]:
%%time
val = alphaBetaMax(gStart, -1, 1)
val

CPU times: total: 0 ns
Wall time: 17 ms


0

When we check the size of the cache we realize that there are many states which we don't have to evaluate any more.

In [24]:
len(gCache)

2747

Let's draw the board.

In [25]:
canvas = create_canvas()
draw(gStart, canvas, val)

Canvas()

Now its time to play.  In the input window that will pop up later, enter your move in the format "`row, count`".  Here `row` is the row that you choose.  The counting of the row  starts form 0. `count` is the number of matches that you want to take from the specified row. 

In [ ]:
play_game(canvas)